In [1]:
from random import randint

class Path():
    def __init__(self,name):
        self.name = name
        self.touched = 0

class RandomScheduler():
        
    def setup(self,total):
        self.total   = total
        self.offset  = 0
        self.margins = []
        self.pathes  = []
    
    def insert(self,path,target):
        self.pathes.append(path)
        self.margins.append(self.offset + target)
        self.offset += target
    
    def get_next_path(self):
        index = 0
        r = randint(0,self.total)
        for margin in self.margins:
            if(r <= margin):
                return self.pathes[index]
            index += 1
        
_NODE_ID_ = 0
class Node():
    def __init__(self,value):
        global _NODE_ID_
        _NODE_ID_    += 1
        self.id       = _NODE_ID_
        self.left     = None
        self.right    = None
        self.next     = None
        self.path     = None
        self.remained = value
        self.occupy   = 0
        
    def set_path(self,path):
        if(self.right != None or self.left != None):
            raise Exception('Path can not be assigned to a node, in which left or right node are assigned')
        self.path = path
    def get_path(self):
        return self.path

    def set_remained(self,value):
        self.remained = value
    def get_remained(self):
        return self.remained
    
    
    def get_next(self):
        if(self.next is None):
            self.next = self.left
            return self.left
        if(self.next is self.left) : 
            self.next = self.right
            return self.left
        else:
            self.next = self.left
            return self.right
    
    def __str__(self):
        return "Left: " + (" " if self.left is not None else "Not ") + "Exists | " + "Right: " + (" " if self.left is not None else "Not ") + "Exists" 
        

class SchedulerTree():

    def __init__(self,max_level):
        self.max_level = max_level
    
    def setup(self,total_value):
        self.root  = Node(total_value)
        self.total = total_value
        self.margin = total_value / (2 ** self.max_level)
    
    def insert(self, path, value):
        self._insert(self.root,path,[value],self.total)
        
    def _insert(self, node, path, value, level_value):
        dvalue = 0
        right_level_value = 0
        left_level_value = 0
        
        if(node.remained == 0 or value[0] < 1):
            return dvalue
        
        if((node.remained <= value[0] or value[0] < self.margin) and node.left == None and node.right == None):
            value[0]      -= node.remained
            dvalue         = node.remained
            node.occupy    = node.remained            
            node.path      = path            
            node.remained -= dvalue
            return dvalue
        
        left_level_value = level_value / 2
        if(node.left == None):
            node.left = Node(left_level_value)
            node.next = node.left
        
        if(0 < node.left.remained):
            dvalue += self._insert(node.left, path, value, left_level_value)
        
        if(value[0] < 1):
            node.remained -= dvalue
            return dvalue
        
        right_level_value = level_value / 2
        if(node.right == None):
            node.right = Node(right_level_value)
        
        if(0 < node.right.remained):
            dvalue += self._insert(node.right, path, value, right_level_value)
        
        return dvalue
    
    def get_next_path(self):
        selected = self.root
        while(selected.left is not None and selected.right is not None):
            selected = selected.get_next()
        if(selected is None or selected.path is None):
            return "No path is selected"
        return selected.path
    
    def __str__(self):
        print("BEGIN")
        print("Total: " + str(self.total) + " Margin: " + str(self.margin))
        self._printing(self.root,0)
        return "END"
        
    def _printing(self,node,level):
        if node == None:
            return
        print("--" * level + " Node: " + str(node.id) + " Path: " + (node.path.name if node.path != None else "-1") + " Occupy: " + str(node.occupy))
        self._printing(node.left,  level + 1)
        self._printing(node.right, level + 1)
    


In [95]:
from collections import deque
import csv
import time
import numpy as np

class TestCase():
    def __init__(self, runs, target1, target2):
        self.runs    = runs
        self.target1 = target1
        self.target2 = target2
        self.total   = target1 + target2
    
class Tester():
    def __init__(self,p1,p2):
        self.past = deque(maxlen=100)
        self.p1   = p1
        self.p2   = p2
        self.tcs  = []

    def addTestCase(self, tc):
        self.tcs.append(tc)
    
    def do(self, csvs):
        stats = []
        schedulers = [SchedulerTree(7), RandomScheduler()]
        for i in range(0,2):
            csv_name = csvs[i]
            sch      = schedulers[i]
            
            start    = time.time()
            results  = self._do(sch)
            end      = time.time()
            stats.append(end - start)
            with open(csv_name, "wb") as f:
                writer = csv.writer(f)
                writer.writerows(results)
        return stats
            
    
    
    def _do(self, sch):
        results = []
        selects = deque()
        self.p1.touched = 0
        self.p2.touched = 0
        for tc in self.tcs:
            sch.setup(tc.total)
            sch.insert(self.p1, tc.target1)
            sch.insert(self.p2, tc.target2)
            true_r1 = float(tc.target1) / float(tc.total)
            true_r2 = float(tc.target2) / float(tc.total)  
            for i in range(0,tc.runs):
                selected = sch.get_next_path()
                selected.touched += 1
                selects.appendleft(selected)
                if(100 < len(selects)):
                    selected = selects.pop()
                    selected.touched -= 1
                act_r1 = float(self.p1.touched) / float(len(selects))
                act_r2 = float(self.p2.touched) / float(len(selects))  
                e1     = float(abs(true_r1 - act_r1)) / true_r1
                e2     = float(abs(true_r2 - act_r2)) / true_r2                
                results.append([act_r1, true_r1, act_r2, true_r2, e1, e2])
        return results
                
                

In [97]:
path1  = Path("P1")
path2  = Path("P2")
tester = Tester(path1,path2)

tester.addTestCase(TestCase(200, 5000, 5000))
tester.addTestCase(TestCase(200, 9000, 1000))
tester.addTestCase(TestCase(200, 3000, 7000))
tester.addTestCase(TestCase(400, 5000, 5000))

stats = []

for i in range(0,10):
    tree_csv = "tree_points_" + str(i) + ".csv"
    rnd_csv  = "rnd_points_" + str(i) + ".csv"
    stats.append(tester.do([tree_csv, rnd_csv]))

with open("stats.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(stats)

In [98]:
!mv *.csv ../../temp/

for i in range(0,10):
    tree_csv = "../../temp/tree_points_" + str(i) + ".csv"
    rnd_csv  = "../../temp/rnd_points_" + str(i) + ".csv"
    output   = "../../temp/schcomp_" + str(i) + ".pdf"
    
    !gnuplot -e "tree_file='$tree_csv'" \
             -e "rnd_file='$rnd_csv'" \
             -e "output_file='$output'" \
            ../gnuplots/schtree.plot
